In [10]:
from skyfield.api import *
from pytz import timezone
import datetime as dt
from skyfield import almanac
from skyfield import *

def lokasi(latitud, longitud):
    zone = timezone('Asia/Kuala_Lumpur')
    lokasi_tempatan = wgs84.latlon(latitud*N, longitud*E)
    return  lokasi_tempatan
def waktu_maghrib(lokasi, tahun, bulan, hari):
    zone = timezone('Asia/Kuala_Lumpur')
    set_masa = zone.localize(dt.datetime(tahun, bulan, hari))
    set_awal_hari = set_masa.replace(hour= 0, minute= 0, second= 0, microsecond= 0)
    set_akhir_hari = set_awal_hari + dt.timedelta(days = 1)
    
    ts = load.timescale()
    awal_hari = ts.from_datetime(set_awal_hari)
    akhir_hari = ts.from_datetime(set_akhir_hari)
    
    eph = load('de440s.bsp')
    terbenam_matahari = almanac.sunrise_sunset(eph, penang)
    
    waktu_maghrib_datetime, terbenam = almanac.find_discrete(awal_hari, akhir_hari, terbenam_matahari)
    maghrib_skyfield = waktu_maghrib_datetime[terbenam == 0]
    
    maghrib_1 = maghrib_skyfield[0]
    waktu_maghrib1 = str(maghrib_1.astimezone(zone))[11:19]
    tarikh = str(maghrib_1.astimezone(zone))[:11]
    
    return maghrib_1, waktu_maghrib1, tarikh

def altitud_jarak_lengkung_bulan(waktu, lokasi_tempatan):
    planet = load('de440s.bsp')
    bumi, bulan, matahari = planet['earth'], planet['moon'], planet['sun']
    ephemeris = api.load('de440s.bsp')
    lok = bumi + lokasi_tempatan
    lokasi_bulan = lok.at(waktu[0]).observe(bulan)
    lokasi_matahari = lok.at(waktu[0]).observe(matahari)
    jarak_lengkung = lokasi_matahari.separation_from(lokasi_bulan)
    jarak_lengkung_darjah = '%.4f' %jarak_lengkung.degrees
    altitud_bulan, az_bulan, jarak_bulan = lokasi_bulan.apparent().altaz()
    altitud_bulan_darjah = '%.4f' %altitud_bulan.degrees
    
    return jarak_lengkung_darjah, altitud_bulan_darjah
def imkanur_rukyah__syarat_643 (lokasi, tahun_masihi, hijri_hari_pertama, hijri_bulan_pertama):
    hijri_hari = hijri_hari_pertama
    hijri_bulan = hijri_bulan_pertama
    list_hijri_hari = [hijri_hari]
    list_hijri_bulan = [hijri_bulan]
    tarikh_masihi_list = []
    for i in range(1,13):
        for f in range (1,32):
            if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
                continue 
            wak_magh = waktu_maghrib(lokasi, tahun_masihi, i, f)
            tarikh_masihi_list.append(wak_magh[2])
            alt_jar_bulan = altitud_jarak_lengkung_bulan(wak_magh, lokasi)
            jarak_lengkung = float(alt_jar_bulan[0])
            altitud_bulan = float(alt_jar_bulan[1])
            if hijri_hari < 29 and hijri_bulan < 13:
                hijri_hari += 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 30 and hijri_bulan <12:
                hijri_bulan += 1
                hijri_hari = 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 30 and hijri_bulan == 12:
                hijri_hari = 1
                hijri_bulan = 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
            elif hijri_hari == 29 and hijri_bulan < 13:
                if jarak_lengkung < 6.4:
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                elif jarak_lengkung > 6.4 and altitud_bulan < 3:
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                else:
                    if hijri_bulan == 12:
                        hijri_hari = 1
                        hijri_bulan = 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
                    else:
                        hijri_hari = 1
                        hijri_bulan += 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
    return list_hijri_hari, list_hijri_bulan, tarikh_masihi_list

In [43]:
import pandas as pd

lokasi_anda = lokasi(5.33, 100.22)
takwim = imkanur_rukyah__syarat_643(lokasi_anda, 2022, 27, 5)
takwim_hijri = pd.DataFrame(list(zip(takwim[0], takwim[1], takwim[2])), columns=['Hari Hijri','Bulan Hijri', 'Masihi'])
takwim_hijri

In [45]:
takwim.to_csv('takwim1.csv')